# Example 2: Feature Space Exploration

This notebook explores how **feature engineering** shapes the selection process. We chain statistical summaries with PCA dimensionality reduction, then use the full suite of feature-space diagnostics to understand the resulting representation.

Key concepts introduced:
- **FeaturePipeline**: chaining multiple feature engineers
- **PCA**: reducing high-dimensional features to interpretable components
- **Multi-objective selection**: balancing Wasserstein fidelity, correlation preservation, and centroid balance
- **KMedoids cluster-size weights**: non-uniform representation

In [ ]:
import pandas as pd
import energy_repset as rep
import energy_repset.diagnostics as diag

In [ ]:
url = "https://tubcloud.tu-berlin.de/s/pKttFadrbTKSJKF/download/time-series-lecture-2.csv"
df_raw = pd.read_csv(url, index_col=0, parse_dates=True).rename_axis('variable', axis=1)
df_raw = df_raw.drop('prices', axis=1)

slicer = rep.TimeSlicer(unit="month")
context = rep.ProblemContext(df_raw=df_raw, slicer=slicer)
print(f"{len(context.get_unique_slices())} candidate monthly slices")

## Feature engineering: Statistics + PCA

`StandardStatsFeatureEngineer` computes ~28 features per month (7 statistics x 4 variables). That's a lot of dimensions for only 12 data points. `PCAFeatureEngineer` then reduces this to the principal components that capture the most variance — a common and effective way to avoid the curse of dimensionality while retaining the essential structure.

In [ ]:
feature_pipeline = rep.FeaturePipeline(
    engineers={
        'stats': rep.StandardStatsFeatureEngineer(),
        'pca': rep.PCAFeatureEngineer(),
    }
)
context = feature_pipeline.run(context)
context.df_features

### How much variance does each component explain?

In [ ]:
fig = diag.PCAVarianceExplained(feature_pipeline.engineers['pca']).plot(show_cumulative=True)
fig.update_layout(title='PCA Variance Explained')
fig.show()

### Feature space: 2D and 3D scatter

Projecting the 12 months into PCA space reveals their relative similarity. Months that are close together have similar statistical profiles; distant months represent different conditions.

In [ ]:
fig = diag.FeatureSpaceScatter2D().plot(context.df_features, x='pc_0', y='pc_1')
fig.update_layout(title='Feature Space: PC0 vs PC1')
fig.show()

In [ ]:
fig = diag.FeatureSpaceScatter3D().plot(context.df_features, x='pc_0', y='pc_1', z='pc_2')
fig.update_layout(title='Feature Space: 3D PCA Projection')
fig.show()

### Feature correlations and distributions

In [ ]:
fig = diag.FeatureCorrelationHeatmap().plot(context.df_features, method='pearson')
fig.update_layout(title='Feature Correlation Matrix')
fig.show()

In [ ]:
fig = diag.FeatureSpaceScatterMatrix().plot(
    context.df_features, dimensions=['pc_0', 'pc_1', 'pc_2', 'pc_3']
)
fig.update_layout(title='PCA Scatter Matrix')
fig.show()

In [ ]:
fig = diag.FeatureDistributions().plot(context.df_features, nbins=20, cols=4)
fig.update_layout(title='Feature Distributions')
fig.show()

## Define objectives and run the search

We use three objectives to capture different facets of representativeness:

- **Wasserstein fidelity**: do the marginal distributions match?
- **Correlation fidelity**: are cross-variable dependencies preserved?
- **Centroid balance**: is the selection centered in the data cloud (avoiding systematic bias)?

The `ParetoMaxMinStrategy` selects the Pareto-optimal combination that maximizes the *worst-performing* objective — a balanced, conservative choice.

In [ ]:
objective_set = rep.ObjectiveSet({
    'wasserstein': (0.5, rep.WassersteinFidelity()),
    'correlation': (0.5, rep.CorrelationFidelity()),
    'centroid_balance': (0.5, rep.CentroidBalance()),
})

k = 3
search_algorithm = rep.ObjectiveDrivenCombinatorialSearchAlgorithm(
    objective_set,
    rep.ParetoMaxMinStrategy(),
    rep.ExhaustiveCombiGen(k=k),
)
representation_model = rep.KMedoidsClustersizeRepresentation()

In [ ]:
workflow = rep.Workflow(feature_pipeline, search_algorithm, representation_model)
experiment = rep.RepSetExperiment(context, workflow)
result = experiment.run()

print(f"Selected months: {result.selection}")
print(f"Weights: {result.weights}")
print(f"Scores: {result.scores}")

## Result diagnostics

### Responsibility weights

Unlike the uniform model in Example 1, the KMedoids representation assigns weights proportional to how many months each representative "covers". The dashed line shows where the bars would be for uniform weighting.

In [ ]:
fig = diag.ResponsibilityBars().plot(result.weights, show_uniform_reference=True)
fig.update_layout(title='Responsibility Weights (KMedoids Cluster-Size)')
fig.show()

### Selection in feature space

The selected months (highlighted) should be well-spread across the feature space — covering the range of conditions rather than clustering in one corner.

In [ ]:
fig = diag.FeatureSpaceScatter2D().plot(
    context.df_features, x='pc_0', y='pc_1', selection=result.selection
)
fig.update_layout(title='Feature Space with Selected Months')
fig.show()

## Score component diagnostics

These plots assess how well the selected months reproduce specific properties of the full year.

### Distribution fidelity (ECDF)

The empirical CDF of the selection should track the full-year curve closely. Gaps indicate that the selection under- or over-represents certain value ranges.

In [ ]:
selected_indices = context.slicer.get_indices_for_slice_combi(context.df_raw.index, result.selection)
df_selection = context.df_raw.loc[selected_indices]

fig = diag.DistributionOverlayECDF().plot(context.df_raw['load'], df_selection['load'])
fig.update_layout(title='Distribution Fidelity: Demand (ECDF)')
fig.show()

### Correlation preservation

The heatmap shows the *difference* between the correlation matrix of the selection and the full year. Values near zero (light) mean the selection preserves that relationship well.

In [ ]:
fig = diag.CorrelationDifferenceHeatmap().plot(
    context.df_raw, df_selection, method='pearson', show_lower_only=True
)
fig.update_layout(title='Correlation Difference: Selection - Full Year')
fig.show()

### Diurnal profiles

Comparing the average hourly shape (hour 0-23) of each variable between the full year and the selection. Good matches here mean the selection captures typical within-day patterns.

In [ ]:
fig = diag.DiurnalProfileOverlay().plot(
    context.df_raw, df_selection, variables=['load', 'onwind', 'offwind', 'solar']
)
fig.update_layout(title='Diurnal Profiles: Full Year vs Selection')
fig.show()